In [26]:
import pandas as pd
import statsmodels.api as sm



In [28]:
dest = pd.read_excel(r"C:\Users\jacob\OneDrive\Desktop\Fall 2024\Telling Stories\housing_report\report-data-2024\Destinations_10.29.22-10.29.2024.xlsx")
hsf = pd.read_excel(r"C:\Users\jacob\OneDrive\Desktop\Fall 2024\Telling Stories\housing_report\report-data-2024\Housing Solutions Fund.xlsx")
mces = pd.read_excel(r"C:\Users\jacob\OneDrive\Desktop\Fall 2024\Telling Stories\housing_report\report-data-2024\MCES_Whole_1.1.19-10.29.24.xlsx")


In [9]:
print(dest.columns)
print(hsf.columns)
print(mces.columns)

Index(['HMIS ID', 'Entry/Exit Destination', 'Entry Date', 'Exit Date',
       'Days to Destination', 'Veteran', 'Date of Birth', 'Age',
       'Household Type', 'Pregnant', 'Chronicity', 'Race (Retired)',
       'Ethnicity (Retired)'],
      dtype='object')
Index(['HMIS #', 'Race', 'Hispanic/Latino', 'Total paid', 'Funding Source',
       'Date of payment', 'Referral from', 'Paid to:', 'What for?',
       'Prevention or Literally Homeless', 'Household Type', 'Income Range'],
      dtype='object')
Index(['HMIS Client Identifier', 'Active ROI', 'Household Type',
       'Veteran Status', 'Date Identified', 'Chronic Status', 'Pregnant',
       'Substance Use', 'MAP Assessment Score', 'Assessment Date', 'Age',
       'DV in past 2 years', 'Foster Care History',
       'Enrolled Member or First Descendent', 'Which Tribe?', 'Gender',
       'Race and Ethnicity', 'Additional Race and Ethnicity Detail',
       'Last Contact Date', 'vInclude Row - active'],
      dtype='object')


In [10]:
# Standardize column names
dest.rename(columns={'HMIS ID': 'HMIS_ID'}, inplace=True)
hsf.rename(columns={'HMIS #': 'HMIS_ID'}, inplace=True)
mces.rename(columns={'HMIS Client Identifier': 'HMIS_ID'}, inplace=True)

# Merge the DataFrames
big_df = dest.merge(hsf, on='HMIS_ID', how='inner') \
             .merge(mces, on='HMIS_ID', how='inner')

# Check the resulting DataFrame
print(big_df.head())


  HMIS_ID                             Entry/Exit Destination Entry Date  \
0  169580                                     Deceased (HUD) 2022-02-09   
1  169580                                     Deceased (HUD) 2022-02-09   
2  197229  Emergency shelter, including hotel or motel pa... 2023-04-12   
3  195043   Foster care home or foster care group home (HUD) 2023-01-17   
4  162438  Jail, prison, or juvenile detention facility (... 2021-11-16   

   Exit Date  Days to Destination    Veteran Date of Birth  Age_x  \
0 2023-05-30                  475  Yes (HUD)    1958-06-18     66   
1 2023-05-30                  475  Yes (HUD)    1958-06-18     66   
2 2024-03-26                  349   No (HUD)    1972-03-16     52   
3 2023-05-02                  105   No (HUD)    1969-10-06     55   
4 2023-12-08                  752   No (HUD)    1987-05-03     37   

  Household Type_x Pregnant_x  ... Age_y DV in past 2 years  \
0       Individual         No  ...  66.0                Yes   
1       

In [11]:
# Get all distinct values in the 'Entry/Exit Destination' column
distinct_values = big_df['Entry/Exit Destination'].unique()

# Print the distinct values
print(distinct_values)


['Deceased (HUD)'
 'Emergency shelter, including hotel or motel paid for with emergency shelter voucher, Host Home shelter (HUD)'
 'Foster care home or foster care group home (HUD)'
 'Jail, prison, or juvenile detention facility (HUD)'
 'No exit interview completed (HUD)'
 'Place not meant for habitation (e.g., a vehicle, an abandoned building, bus/train/subway station/airport or anywhere outside) (HUD)'
 'Psychiatric hospital or other psychiatric facility (HUD)'
 'Rental by client, no ongoing housing subsidy (HUD)'
 'Rental by client, with ongoing housing subsidy (HUD)'
 'Staying or living with family, permanent tenure (HUD)'
 'Staying or living with family, temporary tenure (e.g., room, apartment, or house) (HUD)'
 'Staying or living with friends, permanent tenure (HUD)'
 'Staying or living with friends, temporary tenure (e.g., room, apartment, or house) (HUD)'
 'Transitional housing for homeless persons (including homeless youth) (HUD)']


In [12]:
# Create the "exit" DataFrame by filtering rows
exit = big_df[
    (big_df['Entry/Exit Destination'] != "No exit interview completed (HUD)") &
    (big_df['Entry/Exit Destination'].notna()) &  # Exclude blank (NaN) values
    (big_df['Entry/Exit Destination'] != "")      # Exclude empty string values
]

# Check the first few rows of the "exit" DataFrame
print(exit.head())


  HMIS_ID                             Entry/Exit Destination Entry Date  \
0  169580                                     Deceased (HUD) 2022-02-09   
1  169580                                     Deceased (HUD) 2022-02-09   
2  197229  Emergency shelter, including hotel or motel pa... 2023-04-12   
3  195043   Foster care home or foster care group home (HUD) 2023-01-17   
4  162438  Jail, prison, or juvenile detention facility (... 2021-11-16   

   Exit Date  Days to Destination    Veteran Date of Birth  Age_x  \
0 2023-05-30                  475  Yes (HUD)    1958-06-18     66   
1 2023-05-30                  475  Yes (HUD)    1958-06-18     66   
2 2024-03-26                  349   No (HUD)    1972-03-16     52   
3 2023-05-02                  105   No (HUD)    1969-10-06     55   
4 2023-12-08                  752   No (HUD)    1987-05-03     37   

  Household Type_x Pregnant_x  ... Age_y DV in past 2 years  \
0       Individual         No  ...  66.0                Yes   
1       

In [18]:
import os

# Define the path to save the CSV file in your repository
repo_path = r'C:\Users\jacob\OneDrive\Desktop\Fall 2024\Telling Stories\housing_report\report-data-2024'  # Replace with the actual path to your repository
file_name = 'exit_data.csv'
file_path = os.path.join(repo_path, file_name)

# Save the dataframe as a CSV file
exit.to_csv(file_path, index=False)

print(f'Dataframe saved to {file_path}')


Dataframe saved to C:\Users\jacob\OneDrive\Desktop\Fall 2024\Telling Stories\housing_report\report-data-2024\exit_data.csv


In [25]:
from scipy.stats import f_oneway


# Focus on "Entry/Exit Destination" and numeric variables
numeric_columns = ['Days to Destination', 'MAP Assessment Score', 'Total paid']
grouped_data = exit.groupby('Entry/Exit Destination')

# Perform ANOVA for each numeric column
for col in numeric_columns:
    groups = [group[col].dropna() for _, group in grouped_data]
    f_stat, p_val = f_oneway(*groups)
    print(f"ANOVA for {col}: F-statistic = {f_stat:.2f}, p-value = {p_val:.5f}")
    if p_val < 0.05:
        print(f"Significant differences found in {col} across 'Entry/Exit Destination' groups.\n")
    else:
        print(f"No significant differences found in {col} across 'Entry/Exit Destination' groups.\n")


ANOVA for Days to Destination: F-statistic = 1.26, p-value = 0.25370
No significant differences found in Days to Destination across 'Entry/Exit Destination' groups.

ANOVA for MAP Assessment Score: F-statistic = nan, p-value = nan
No significant differences found in MAP Assessment Score across 'Entry/Exit Destination' groups.

ANOVA for Total paid: F-statistic = 5.31, p-value = 0.00000
Significant differences found in Total paid across 'Entry/Exit Destination' groups.



C:\Users\jacob\AppData\Local\Temp\ipykernel_5532\3502120461.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_data = exit.groupby('Entry/Exit Destination')
c:\Users\jacob\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\stats\_stats_py.py:4102: DegenerateDataWarning: at least one input has length 0
  if _f_oneway_is_too_small(samples):
